<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/glqp_secure_pipeline_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install cryptography numpy matplotlib seaborn

In [ ]:
pip install qiskit

In [ ]:
#!/usr/bin/env python3
# glqp_secure_pipeline.py
# End-to-end: parameter sweep -> visuals -> metrics -> (optional) quantum view -> encrypted package
# Decrypt mode included.
import argparse
import base64
import csv
import io
import json
import os
import tarfile
import textwrap
from datetime import datetime
from pathlib import Path
from typing import List, Tuple, Dict

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- Crypto (Fernet with PBKDF2) ---
try:
    from cryptography.hazmat.primitives import hashes
    from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
    from cryptography.fernet import Fernet
    CRYPTO_OK = True
except Exception:
    CRYPTO_OK = False

sns.set(style="whitegrid")

# ---------- Visualization and simulation ----------

def generate_phase_profile_plot(phase_profile: np.ndarray, out_path: Path, title: str):
    plt.figure(figsize=(6, 4))
    positions = np.arange(len(phase_profile))
    plt.plot(positions, phase_profile, marker='o', linestyle='-', color='mediumblue')
    plt.title(title)
    plt.xlabel("Gate Position")
    plt.ylabel("Phase Shift (radians)")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(out_path, dpi=150)
    plt.close()

def simulate_interference(phase_profile: np.ndarray,
                          spacing: float,
                          k: float,
                          grid_size: int,
                          extent: Tuple[float, float, float, float]) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    x_min, x_max, y_min, y_max = extent
    x = np.linspace(x_min, x_max, grid_size)
    y = np.linspace(y_min, y_max, grid_size)
    X, Y = np.meshgrid(x, y)
    field = np.zeros_like(X)
    n = len(phase_profile)
    start_x = -spacing * (n - 1) / 2.0
    for i, phase in enumerate(phase_profile):
        x0 = start_x + i * spacing
        r = np.sqrt((X - x0) ** 2 + Y ** 2)
        field += np.cos(k * r + phase)
    return X, Y, field

def save_interference_plot(X: np.ndarray, Y: np.ndarray, field: np.ndarray, out_path: Path, title: str):
    plt.figure(figsize=(6, 5))
    plt.imshow(field, extent=[X.min(), X.max(), Y.min(), Y.max()],
               cmap="plasma", origin="lower", aspect='equal')
    plt.title(title)
    plt.xlabel("X Position")
    plt.ylabel("Y Position")
    cbar = plt.colorbar()
    cbar.set_label("Intensity")
    plt.tight_layout()
    plt.savefig(out_path, dpi=150)
    plt.close()

def extract_central_line(field: np.ndarray, Y: np.ndarray) -> np.ndarray:
    # Choose the row whose Y is closest to 0
    idx = np.abs(Y[:, 0]).argmin()
    return field[idx, :]

def compute_visibility(intensity_line: np.ndarray) -> float:
    Imax = float(np.max(intensity_line))
    Imin = float(np.min(intensity_line))
    denom = Imax + Imin
    if denom == 0:
        return 0.0
    return (Imax - Imin) / denom

# ---------- Optional quantum circuit view (if Qiskit is available) ----------

def try_quantum_view(phase_profile: np.ndarray,
                     cp_strengths: List[float],
                     out_dir: Path,
                     log_lines: List[str]):
    try:
        from qiskit import QuantumCircuit
        from qiskit.circuit.library import QFT
        from qiskit.quantum_info import Statevector
        has_qiskit = True
    except Exception as e:
        has_qiskit = False
        log_lines.append(f"[QISKIT] Not available: {e}")

    if not has_qiskit:
        # Write a short note so the pipeline remains transparent
        note_path = out_dir / "quantum_view_skipped.txt"
        note_path.write_text("Qiskit not installed; quantum circuit view skipped.\n"
                             "Install with: pip install qiskit\n")
        return

    n = len(phase_profile)
    for cp in cp_strengths:
        qc = QuantumCircuit(n, name=f"QFT_phase_QFTdg_CP{cp}")
        # Hadamards to create superposition-like aperture
        for q in range(n):
            qc.h(q)
        # QFT
        qc.append(QFT(num_qubits=n, do_swaps=True).to_gate(label="QFT"), range(n))
        # Per-qubit phases
        for q, phi in enumerate(phase_profile):
            qc.p(phi, q)
        # Inverse QFT
        qc.append(QFT(num_qubits=n, do_swaps=True, inverse=True).to_gate(label="QFT†"), range(n))
        # Chain of controlled phases to add path correlations
        for q in range(n - 1):
            qc.cp(cp, q, q + 1)

        # Statevector simulation
        sv = Statevector.from_instruction(qc)
        probs = np.abs(sv.data) ** 2
        # Plot probability bar chart
        fig, ax = plt.subplots(figsize=(7, 4))
        x = np.arange(len(probs))
        ax.bar(x, probs, color="teal", alpha=0.9)
        ax.set_title(f"QFT–Phase–QFT† with CP={cp:.2f} | {phase_profile.tolist()}")
        ax.set_xlabel("Computational Basis State Index")
        ax.set_ylabel("Probability")
        ax.set_ylim(0, max(0.25, probs.max() * 1.1))
        plt.tight_layout()
        out_path = out_dir / f"quantum_view_probs_cp{str(cp).replace('.','_')}.png"
        plt.savefig(out_path, dpi=150)
        plt.close()

        # Save circuit diagram (text)
        circ_path = out_dir / f"quantum_view_circuit_cp{str(cp).replace('.','_')}.txt"
        circ_path.write_text(qc.draw(output="text").single_string())

# ---------- Packaging and encryption ----------

def derive_fernet_key(passphrase: str, salt: bytes) -> bytes:
    # PBKDF2 with SHA-256, 390k iterations (tunable), 32-byte key for Fernet
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=390000,
    )
    key = kdf.derive(passphrase.encode("utf-8"))
    return base64.urlsafe_b64encode(key)

def package_artifacts_to_tar_gz(artifacts_dir: Path) -> bytes:
    buf = io.BytesIO()
    with tarfile.open(fileobj=buf, mode="w:gz") as tf:
        # Add all files under artifacts_dir
        for root, _, files in os.walk(artifacts_dir):
            for f in files:
                full_path = Path(root) / f
                arcname = full_path.relative_to(artifacts_dir)
                tf.add(full_path, arcname=str(arcname))
    buf.seek(0)
    return buf.read()

def encrypt_payload(payload: bytes, passphrase: str) -> Dict[str, str]:
    if not CRYPTO_OK:
        raise RuntimeError("cryptography not installed. Install with: pip install cryptography")
    import secrets
    salt = secrets.token_bytes(16)
    key = derive_fernet_key(passphrase, salt)
    token = Fernet(key).encrypt(payload)
    return {
        "format": "glqp.enc.v1",
        "salt_b64": base64.urlsafe_b64encode(salt).decode("ascii"),
        "token_b64": base64.urlsafe_b64encode(token).decode("ascii"),
        "created_utc": datetime.utcnow().isoformat() + "Z",
    }

def decrypt_payload(enc_obj: Dict[str, str], passphrase: str) -> bytes:
    if not CRYPTO_OK:
        raise RuntimeError("cryptography not installed. Install with: pip install cryptography")
    if enc_obj.get("format") != "glqp.enc.v1":
        raise ValueError("Unsupported encrypted format")
    salt = base64.urlsafe_b64decode(enc_obj["salt_b64"].encode("ascii"))
    token = base64.urlsafe_b64decode(enc_obj["token_b64"].encode("ascii"))
    key = derive_fernet_key(passphrase, salt)
    return Fernet(key).decrypt(token)

# ---------- Pipeline ----------

def preset_profiles() -> Dict[str, List[float]]:
    return {
        "antisymmetric": [-0.375, -0.125, 0.125, 0.375],
        "center_mass_bipolar": [-0.6, 0.0, 0.0, 0.6],
        "cluster_soft": [-0.3, -0.1, 0.1, 0.3],
        "edge_dominant": [-0.8, -0.2, 0.2, 0.8],
        "uniform_small": [0.1, 0.1, 0.1, 0.1],
        "random_seed42": [float(x) for x in np.round(np.linspace(-0.4, 0.4, 4) + 0.05*np.random.default_rng(42).normal(size=4), 3)]
    }

def run_pipeline(workdir: Path,
                 ks: List[float],
                 spacing: float,
                 grid_size: int,
                 cp_strengths: List[float]) -> Tuple[Path, List[str]]:
    timestamp = datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
    artifacts_dir = workdir / f"artifacts_{timestamp}"
    artifacts_dir.mkdir(parents=True, exist_ok=True)
    log_lines = []
    log_lines.append(f"[INFO] Artifacts dir: {artifacts_dir}")

    # Save config for traceability
    (artifacts_dir / "config.json").write_text(json.dumps({
        "ks": ks,
        "spacing": spacing,
        "grid_size": grid_size,
        "cp_strengths": cp_strengths,
        "timestamp_utc": timestamp
    }, indent=2))

    # Prepare CSV tracker
    csv_path = artifacts_dir / "tracker_metrics.csv"
    with open(csv_path, "w", newline="") as fcsv:
        writer = csv.writer(fcsv)
        writer.writerow([
            "profile_name", "phases_json", "k", "spacing",
            "grid_size", "visibility_y0", "central_line_png", "phase_plot_png", "interference_png"
        ])

        # Iterate profiles and ks
        for pname, phases in preset_profiles().items():
            phase_arr = np.array(phases, dtype=float)
            # Phase profile plot
            phase_plot_path = artifacts_dir / f"{pname}_phase_profile.png"
            generate_phase_profile_plot(phase_arr, phase_plot_path, f"Phase Profile: {pname}")

            for k in ks:
                X, Y, field = simulate_interference(phase_arr, spacing=spacing, k=k,
                                                    grid_size=grid_size, extent=(-2, 2, -2, 2))
                inter_path = artifacts_dir / f"{pname}_interference_k{k:.0f}.png"
                save_interference_plot(X, Y, field, inter_path, f"Lensing Interference: {pname}, k={k}")

                # 1D central line cut
                central = extract_central_line(field, Y)
                vis = compute_visibility(central)

                # Save central line plot
                fig, ax = plt.subplots(figsize=(6, 3.5))
                ax.plot(np.linspace(-2, 2, field.shape[1]), central, color="crimson")
                ax.set_title(f"Central-Line Intensity: {pname}, k={k} (V={vis:.3f})")
                ax.set_xlabel("X Position (Y=0)")
                ax.set_ylabel("Intensity")
                ax.grid(True, alpha=0.4)
                plt.tight_layout()
                central_png = artifacts_dir / f"{pname}_central_line_k{k:.0f}.png"
                plt.savefig(central_png, dpi=150)
                plt.close()

                # Write CSV row
                writer.writerow([
                    pname, json.dumps(phases), k, spacing, grid_size, f"{vis:.6f}",
                    central_png.name, phase_plot_path.name, inter_path.name
                ])

    # Quantum circuit grounded view (optional)
    qview_dir = artifacts_dir / "quantum_view"
    qview_dir.mkdir(exist_ok=True)
    try_quantum_view(np.array(preset_profiles()["antisymmetric"]), cp_strengths, qview_dir, log_lines)

    # Save log
    (artifacts_dir / "pipeline.log").write_text("\n".join(log_lines) + "\n")
    return artifacts_dir, log_lines

def run_encrypt(artifacts_dir: Path, passphrase: str, out_path: Path) -> Path:
    payload = package_artifacts_to_tar_gz(artifacts_dir)
    enc_obj = encrypt_payload(payload, passphrase)
    out_path.write_text(json.dumps(enc_obj, indent=2))
    # Optionally, remove the plaintext tar (we didn't write a tar file to disk; encryption uses in-memory bytes)
    # Leave artifacts on disk for transparency; if you want, you can clean them after verifying the .enc.json
    # Write a short README with decrypt instructions
    readme = textwrap.dedent(f"""
    Encrypted package created: {out_path}
    Decrypt with:
      python glqp_secure_pipeline.py --decrypt --encrypted "{out_path}" --passphrase "YOUR-PASSPHRASE" --outdir "./decrypted_output"
    """).strip()
    (artifacts_dir / "ENCRYPTION_README.txt").write_text(readme + "\n")
    return out_path

def run_decrypt(enc_path: Path, passphrase: str, outdir: Path):
    enc_obj = json.loads(enc_path.read_text())
    payload = decrypt_payload(enc_obj, passphrase)
    outdir.mkdir(parents=True, exist_ok=True)
    # Extract tar.gz payload
    with tarfile.open(fileobj=io.BytesIO(payload), mode="r:gz") as tf:
        tf.extractall(path=outdir)
    (outdir / "DECRYPT_OK.txt").write_text("Decryption successful.\n")

# ---------- CLI ----------

def main():
    parser = argparse.ArgumentParser(description="GLQP secure pipeline: sweep, visualize, metrics, and encrypted packaging")
    parser.add_argument("--workdir", type=Path, default=Path("/mnt/data/gravitational_lensing"),
                        help="Directory to store artifacts and encrypted package")
    parser.add_argument("--run-all", action="store_true", help="Run parameter sweep, visuals, metrics, and quantum view")
    parser.add_argument("--k", type=float, nargs="+", default=[10.0],
                        help="Wavenumber(s) for interference simulation")
    parser.add_argument("--spacing", type=float, default=1.0, help="Source spacing across x-axis")
    parser.add_argument("--grid-size", type=int, default=400, help="Grid resolution for interference map")
    parser.add_argument("--cp-strengths", type=float, nargs="+", default=[0.0, 0.3, 0.6],
                        help="Controlled-phase strengths for quantum view")
    parser.add_argument("--passphrase", type=str, default=None, help="Passphrase for encryption/decryption")
    parser.add_argument("--encrypted", type=Path, help="Path to encrypted .enc.json package for decryption")
    parser.add_argument("--decrypt", action="store_true", help="Decrypt an encrypted package")
    parser.add_argument("--outdir", type=Path, help="Output directory for decrypted contents")
    args = parser.parse_args()

    if args.decrypt:
        if args.passphrase is None:
            raise SystemExit("ERROR: --passphrase is required for --decrypt")
        if not args.encrypted:
            raise SystemExit("ERROR: --encrypted path is required for --decrypt")
        outdir = args.outdir or (args.workdir / "decrypted")
        run_decrypt(args.encrypted, args.passphrase, outdir)
        print(f"[OK] Decrypted into: {outdir}")
        return

    if args.run_all:
        artifacts_dir, log_lines = run_pipeline(
            workdir=args.workdir,
            ks=args.k,
            spacing=args.spacing,
            grid_size=args.grid_size,
            cp_strengths=args.cp_strengths
        )
        # Encrypt artifacts if passphrase provided
        if args.passphrase:
            enc_out = args.workdir / "glqp_artifacts.enc.json"
            run_encrypt(artifacts_dir, args.passphrase, enc_out)
            print(f"[OK] Artifacts encrypted to: {enc_out}")
        else:
            print("[NOTE] No passphrase provided; artifacts left unencrypted.")
        print(f"[OK] Artifacts directory: {artifacts_dir}")
        return

    # If neither run nor decrypt specified
    parser.print_help()

if __name__ == "__main__":
    main()